In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import random
import datetime
import matplotlib.pyplot as plt
import re
%run functions.ipynb
from unidecode import unidecode
plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option("display.precision", 3)

get_avg(Player,games=26,categories=None)
get_stat(Player,category,stats=pd.DataFrame([])):
get_line(player,pp_stat)
moving_avg(Player,cat,window)
print_prob(player,series,line,cat,games=None,prnt=True)
graph_stat(player,cat,window)
dynamic(guy,cat,window=10)
get_game(player,date)
adjust(player,cat,opp)
best_odds(lines)
check(lines)
get_pos(guy)
season_stats(player)
position_diff(opp,cat)


In [2]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.min_rows', 30)

In [3]:
opp_stats = pd.read_csv('opponent_stats',index_col=0)
schedule = pd.read_csv('schedule',index_col=0)
data = pd.read_csv('data.csv')

lines = pd.read_csv('betting_lines_1',index_col=0)
lines2 =pd.read_csv('betting_lines_2',index_col=0)
lines3 =pd.read_csv('betting_lines_3',index_col=0)
lines4 =pd.read_csv('betting_lines_4',index_col=0)   
lines5 =pd.read_csv('betting_lines_5',index_col=0)   

In [4]:
data.head(3)

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,W (+9),1,38.633,...,12,4,1,1,3,1,35,30.1,10.0,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,W (+7),1,36.783,...,5,4,1,2,2,1,29,22.7,-1.0,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,W (+6),1,37.650,...,8,1,1,2,1,2,40,34.2,6.0,PF


In [20]:
WL = data['W/L'].to_list()
margin = []
for i in range(len(WL)):
    input_string = WL[i]
    output = re.match(r"^[WL]\s*\(([-]?\d+|\+\d+)\)", input_string).group(1)
    margin.append(output)

margin[0:10]
data['W/L']=margin

In [6]:

POSIT = data['Pos'].to_list()
pos_dict = {'PG':1, 'SG': '2','SF':3,'PF':4,'C':5}
pos_i = []
for i in range(len(POSIT)):
    pos_i.append(pos_dict[POSIT[i]])
    
pos_i[0:10]

    

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [7]:
data['Pos']=pos_i
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,W (+9),1,38.633,...,12,4,1,1,3,1,35,30.1,10.0,4
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,W (+7),1,36.783,...,5,4,1,2,2,1,29,22.7,-1.0,4
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,W (+6),1,37.650,...,8,1,1,2,1,2,40,34.2,6.0,4
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,L (-18),1,35.433,...,8,5,0,0,1,2,26,20.7,-8.0,4
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,L (-9),1,42.183,...,7,4,1,1,4,4,32,24.1,-10.0,4
5,Jayson Tatum,6,2022-10-30,24-241,BOS,1,WAS,W (+18),1,27.417,...,4,3,1,0,4,0,23,16.0,10.0,4
6,Jayson Tatum,7,2022-11-02,24-244,BOS,0,CLE,L (-1),1,42.867,...,12,6,1,4,6,0,26,21.3,-16.0,4
7,Jayson Tatum,8,2022-11-04,24-246,BOS,1,CHI,W (+4),1,40.050,...,12,6,1,1,1,4,36,29.6,11.0,4
8,Jayson Tatum,9,2022-11-05,24-247,BOS,0,NYK,W (+15),1,36.517,...,4,5,1,2,3,3,26,20.2,12.0,4
9,Jayson Tatum,10,2022-11-07,24-249,BOS,0,MEM,W (+3),1,39.583,...,3,2,0,2,2,2,39,25.6,15.0,4


In [8]:
acron = data['Opp'].unique()
acron.sort()
acr2i = {a:i for i,a in enumerate(acron)}
acr2i

{'ATL': 0,
 'BOS': 1,
 'BRK': 2,
 'CHI': 3,
 'CHO': 4,
 'CLE': 5,
 'DAL': 6,
 'DEN': 7,
 'DET': 8,
 'GSW': 9,
 'HOU': 10,
 'IND': 11,
 'LAC': 12,
 'LAL': 13,
 'MEM': 14,
 'MIA': 15,
 'MIL': 16,
 'MIN': 17,
 'NOP': 18,
 'NYK': 19,
 'OKC': 20,
 'ORL': 21,
 'PHI': 22,
 'PHO': 23,
 'POR': 24,
 'SAC': 25,
 'SAS': 26,
 'TOR': 27,
 'UTA': 28,
 'WAS': 29}

In [10]:
Opp = data['Opp'].to_list()
tm = data['Tm'].to_list()
opp_i = []
tm_i = []
for i in range(len(Opp)):
    opp_i.append(acr2i[Opp[i]])
    tm_i.append(acr2i[tm[i]])
    
data['Tm'] = tm_i
data['Opp'] = opp_i

In [11]:
data.head()

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,1,1,22,W (+9),1,38.633,...,12,4,1,1,3,1,35,30.1,10.0,4
1,Jayson Tatum,2,2022-10-21,24-232,1,0,15,W (+7),1,36.783,...,5,4,1,2,2,1,29,22.7,-1.0,4
2,Jayson Tatum,3,2022-10-22,24-233,1,0,21,W (+6),1,37.650,...,8,1,1,2,1,2,40,34.2,6.0,4
3,Jayson Tatum,4,2022-10-24,24-235,1,0,3,L (-18),1,35.433,...,8,5,0,0,1,2,26,20.7,-8.0,4
4,Jayson Tatum,5,2022-10-28,24-239,1,1,5,L (-9),1,42.183,...,7,4,1,1,4,4,32,24.1,-10.0,4


In [12]:
data = data.drop(columns=['Age','GmSc','+/-'])

In [13]:
data.columns

Index(['Player', 'G', 'Date', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Pos'],
      dtype='object')

In [14]:
data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
data['Rest'] = data.groupby('Player')['Date'].diff().dt.days
data.head(3)

,Player,G,Date,Tm,H/A,Opp,W/L,GS,MP,FG,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Pos,Rest
0,Jayson Tatum,1,2022-10-18,1,1,22,W (+9),1,38.633,13,...,12,12,4,1,1,3,1,35,4,NaN
1,Jayson Tatum,2,2022-10-21,1,0,15,W (+7),1,36.783,10,...,3,5,4,1,2,2,1,29,4,3.0
2,Jayson Tatum,3,2022-10-22,1,0,21,W (+6),1,37.650,14,...,8,8,1,1,2,1,2,40,4,1.0


In [15]:
data= data.drop(columns=['Date'])

In [16]:
players = data['Player']
p2i = {p:i for i,p in enumerate(players.unique())}

play_i = []
for i in range(len(players)):
    play_i.append(p2i[players[i]])
data['Player'] = play_i

In [100]:
data = data.fillna(2)

KeyError: 0

In [36]:
data = data.astype('float64')

In [70]:
tens = torch.tensor(data.values)

In [103]:
tens[2]

tensor([ 0.0000,  3.0000,  1.0000,  0.0000, 21.0000,  6.0000,  1.0000, 37.6500,
        14.0000, 21.0000,  0.6670,  4.0000, 10.0000,  0.4000,  8.0000,  9.0000,
         0.8890,  0.0000,  8.0000,  8.0000,  1.0000,  1.0000,  2.0000,  1.0000,
         2.0000, 40.0000,  4.0000,  1.0000], dtype=torch.float64)

In [92]:
tens.shape[0]

18821

In [155]:
tens[-5:]

tensor([[500.0000,   1.0000,  19.0000,   1.0000,   8.0000,  24.0000,   0.0000,
           0.9500,   0.0000,   1.0000,   0.0000,   0.0000,   1.0000,   0.0000,
           0.0000,   0.0000,   2.0000,   0.0000,   1.0000,   1.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   2.0000,   2.0000],
        [501.0000,   1.0000,   3.0000,   1.0000,   5.0000, -32.0000,   0.0000,
           1.7333,   0.0000,   1.0000,   0.0000,   0.0000,   1.0000,   0.0000,
           0.0000,   0.0000,   2.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   5.0000,   2.0000],
        [501.0000,   2.0000,   3.0000,   0.0000,   8.0000,  18.0000,   0.0000,
           1.1000,   0.0000,   0.0000,   2.0000,   0.0000,   0.0000,   2.0000,
           0.0000,   0.0000,   2.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   5.0000,  89.0000],
        [501.0000,   3.0000,   3.0000,   1.0000, 

In [158]:
tens.flip(0)[1]

tensor([501.0000,   3.0000,   3.0000,   1.0000,  16.0000, -12.0000,   0.0000,
          1.8000,   0.0000,   0.0000,   2.0000,   0.0000,   0.0000,   2.0000,
          0.0000,   0.0000,   2.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   5.0000,  28.0000],
       dtype=torch.float64)

In [146]:
xs = torch.zeros(503,82,28)#P,G,STATS

for p in range(10):
    print(p)
    for i in tens:
        if i[0] == p:
            xs[p,82-int((i[1]))]= i
        


0
1
2
3
4
5
6
7
8
9


In [150]:
xs[0,81]

tensor([ 0.0000,  1.0000,  1.0000,  1.0000, 22.0000,  9.0000,  1.0000, 38.6333,
        13.0000, 20.0000,  0.6500,  2.0000,  7.0000,  0.2860,  7.0000,  9.0000,
         0.7780,  0.0000, 12.0000, 12.0000,  4.0000,  1.0000,  1.0000,  3.0000,
         1.0000, 35.0000,  4.0000,  2.0000])

In [111]:
xs.shape

torch.Size([503, 82, 28])

In [142]:
a = torch.randint(503,(1,))
xs[a]

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [247.,   1.,  11.,  ...,   3.,   5.,   2.],
         [247.,   2.,  11.,  ...,  16.,   5.,   2.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]]])

In [ ]:
a = torch.randint

In [138]:
model = nn.Sequential(
    nn.Linear(28,1000),
    nn.ReLU(),
    nn.Linear(1000,28)
)

In [139]:
model

Sequential(
  (0): Linear(in_features=28, out_features=1000, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1000, out_features=28, bias=True)
)

In [ ]:
 criterion 